# 学習モデルを使って実際のレース予想をする

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

### 予想したいレースのURLを以下に入力　ここのサイトから選ぶ　https://keirin.kdreams.jp/

In [7]:
url = "https://keirin.kdreams.jp/shizuoka/racedetail/3820210903030011/"

In [8]:
columns = ['予想', '好気合', '総評', '枠番', '車番', '選手名府県/年齢/期別', '級班', '脚質', 'ギヤ倍数', '競走得点', '1着', '2着', '3着', '着外']

### データを取ってくる

In [49]:
race_data = pd.read_html(url)[4][:-1]
race_data.columns = columns
race_data = race_data.astype(str)
time.sleep(1)

### データを加工する

In [50]:
race_data = pd.concat([race_data, race_data["選手名府県/年齢/期別"].str.split("/", expand=True)], axis=1).drop("選手名府県/年齢/期別", axis=1)
race_data = race_data.drop(0, axis=1)

race_data["予想"] = pd.Categorical(race_data["予想"], categories=["nan", "×", "▲", "△", "○", "◎", "注"])
race_data["脚質"] = pd.Categorical(race_data["脚質"], categories=["両", "追", "逃"])
race_data["級班"] = pd.Categorical(race_data["級班"], categories=["A1", "A2", "A3", "L1", "S1", "S2", "SS"])

prediction = pd.get_dummies(race_data["予想"])
isFine = pd.get_dummies(race_data["好気合"], drop_first=True)
leg_type = pd.get_dummies(race_data["脚質"])
group = pd.get_dummies(race_data["級班"])

race_data = pd.concat([race_data, prediction, isFine, leg_type, group], axis=1).drop(["予想", "好気合", "脚質", "級班"], axis=1)
race_data = race_data.replace(["失", "落", "故", "欠"], 9)
race_data["ギヤ倍数"] = race_data["ギヤ倍数"].map(lambda x: x[:4] if len(x)>4 else x)
race_data[2] = race_data[2].map(lambda x: x.replace(" （欠車）", "") if "欠車"in x else x)
race_data = race_data.astype("float64")
race_data = race_data.loc[:, ['車番', '総評', '枠番', 'ギヤ倍数', '競走得点', '1着', '2着', '3着', '着外', 1, 2, 'nan', '×', '▲', '△', '○', '◎', '注', '★', '両', '追', '逃', 'A1', 'A2', 'A3', 'L1', 'S1', 'S2', 'SS']]
race_data

,車番,総評,枠番,ギヤ倍数,競走得点,1着,2着,3着,着外,1,...,両,追,逃,A1,A2,A3,L1,S1,S2,SS
0,1.0,2.0,1.0,3.93,94.47,11.0,7.0,1.0,4.0,24.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2.0,2.0,2.0,3.85,90.93,7.0,5.0,4.0,17.0,33.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,2.0,3.0,3.93,92.26,2.0,2.0,4.0,11.0,47.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,2.0,4.0,3.86,92.91,14.0,13.0,0.0,17.0,32.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,3.0,5.0,3.92,87.19,7.0,3.0,0.0,5.0,24.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,6.0,3.0,6.0,3.92,85.23,6.0,4.0,4.0,25.0,37.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,7.0,2.0,6.0,3.93,89.00,9.0,13.0,8.0,40.0,31.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### 学習済みモデルに入れて予想する

In [51]:
import tensorflow as tf
from tensorflow import keras

In [52]:
model = tf.keras.models.load_model("pred_top3")

In [53]:
race_data.shape

(7, 29)

In [54]:
results = model.predict(race_data)

In [55]:
results

array([[0.9276395 ],
       [0.7401427 ],
       [0.6149575 ],
       [0.5573057 ],
       [0.46025407],
       [0.17545676],
       [0.26905322]], dtype=float32)